In [1]:
!pip install --user albumentations

# !pip install -U albumentations --no-binary qudida,albumentations
# print(albumentations.__version__)
# !pip3 install -q opencv-python==4.5.5.64
# !pip3 install -q --force-reinstall albumentations==1.0.3
# import albumentations
# import albumentations as A
# import cv2
# print(albumentations.__version__)

     -------------------------------------- 116.7/116.7 kB 6.7 MB/s eta 0:00:00
     ---------------------------------------- 35.5/35.5 MB 9.9 MB/s eta 0:00:00


In [1]:
# Modul calling
import cv2
import os
import numpy as np
from PIL import Image

In [2]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def myFig(img, label):
    # Crate figure and axes
    fig, ax = plt.subplots()
    # Display the image
    ax.imshow(img)
    
    if len(label)>0:
        # Create a rectangle patch
        dw = img.shape[1]
        dh = img.shape[0]
        
        x1 = (label[0][0] - label[0][2]/2) * dw
        y1 = (label[0][1] - label[0][3]/2) * dh
        w = label[0][2] * dw
        h = label[0][3] * dh
        rect = patches.Rectangle((x1, y1), w, h, linewidth=1, edgecolor='r', facecolor='none')
        
        # Add the patch to the axes
        ax.add_patch(rect)
    plt.show()

In [3]:
# Module 
import albumentations as A
from albumentations.pytorch import ToTensorV2 
import skimage
from PIL import Image

C:\Users\yuri\anaconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\yuri\anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


### 1. LongestMaxSize

In [13]:
def LongestMaxSize(image, bboxes, IMAGE_SIZE=640):

    train_transforms = A.Compose(
        [
            A.LongestMaxSize(max_size=IMAGE_SIZE),
        ],
        bbox_params=A.BboxParams(format="yolo", min_visibility=0.4, label_fields=[],),
    )
    
    transformed = train_transforms(image=image, bboxes=bboxes)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']
    return(transformed_image, transformed_bboxes)

### 2. ColorJitter
- How to change the color of an image

In [4]:
def ColorJitter(image, bboxes):
    
    train_transforms = A.Compose(
        [
            A.ColorJitter(brightness= 0.6, contrast=0.6, saturation=0.6, hue=0.6, p=1),       
        ],
        bbox_params = A.BboxParams(format="yolo", min_visibility=0.4, label_fields=[],),
    )
    
    transformed = train_transforms(image=image, bboxes=bboxes)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']
    return(transformed_image, transformed_bboxes)  

### 3. ShiftScaleRotate
- How to rotate an image

In [5]:
def ShiftScaleRotate(image, bboxes):

    train_transforms = A.Compose(
        [
            A.ShiftScaleRotate(rotate_limit=50, p=1, border_mode=cv2.BORDER_CONSTANT),       
        ],
        bbox_params = A.BboxParams(format="yolo", min_visibility=0.4, label_fields=[],),
    )
    
    transformed = train_transforms(image=image, bboxes=bboxes)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']
    return(transformed_image, transformed_bboxes)  

### 4. HorizontalFlip
- How to flip an image left and right

In [6]:
def HorizontalFlip(image, bboxes):
    
    train_transforms = A.Compose(
        [
            A.HorizontalFlip(p=1),       
        ],
        bbox_params = A.BboxParams(format="yolo", min_visibility=0.4, label_fields=[],),
    )
    
    transformed = train_transforms(image=image, bboxes=bboxes)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']
    return(transformed_image, transformed_bboxes) 


### 5. Blur
- How to handle blur of image

In [7]:
def Blur(image, bboxes):
    
    train_transforms = A.Compose(
        [
            A.Blur(p=1),       
        ],
        bbox_params = A.BboxParams(format="yolo", min_visibility=0.4, label_fields=[],),
    )
    
    transformed = train_transforms(image=image, bboxes=bboxes)
    transformed_image = transformed['image']
    transformed_bboxes = transformed['bboxes']
    return(transformed_image, transformed_bboxes) 

### Save image and bbox

In [8]:
def SaveImage_Bbox(image, bboxes, BASE_PATH, file_name, augmentation_name=""):
    cv2.imwrite(BASE_PATH+"images/"+file_name + augmentation_name + ".jpg", image)
    bboxes = list(bboxes)
    #print(bboxes)
    
    if sum(bboxes) > 0:
        with open(BASE_PATH+"labels/"+file_name+augmentation_name+".txt", "w") as file:
            file.write((' ').join(list(map(str, [round(bboxes[-1])]+bboxes[:-1]))))
    #print([bboxes[-1]]+bboxes[:-1])

### Main

In [9]:
# Image and label data read calling
BASE_PATH = " " # Training folder or file path for image augmentation + /
error_list = []

for image_name in os.listdir(BASE_PATH + "images/"):
    file_name, ext = os.path.splitext(image_name)
    if ext == ".jpg": # Only image
        img_path = BASE_PATH + "images/" + image_name 
        label_path = BASE_PATH + "labels/" + file_name + ".txt"
        image = cv2.imread(img_path)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        try:
            bboxes = np.loadtxt(fname=label_path, delimiter=" ", ndmin=2)
            bboxes = np.roll(bboxes, 4, axis=1).tolist()
        
            #image, bboxes = LongestMaxSize(image, bboxes)
            #print(bboxes)
            #SaveImage_Bbox(image, bboxes[0], BASE_PATH, file_name)

            CJ_img, CJ_bboxes = ColorJitter(image, bboxes)
            print(CJ_bboxes)
            SaveImage_Bbox(CJ_img, CJ_bboxes[0], BASE_PATH, file_name, "_CJ")

            #SSR_img, SSR_bboxes = ShiftScaleRotate(image, bboxes)
            #print(SSR_bboxes)
            #SaveImage_Bbox(SSR_img, SSR_bboxes[0], BASE_PATH, file_name, "_ShiftScaleRotate")

            HF_img, HF_bboxes = HorizontalFlip(image, bboxes)
            print(HF_bboxes)
            SaveImage_Bbox(HF_img, HF_bboxes[0], BASE_PATH, file_name, "_HF")

            Blur_img, Blur_bboxes = Blur(image, bboxes)
            print(Blur_bboxes)
            SaveImage_Bbox(Blur_img, Blur_bboxes[0], BASE_PATH, file_name, "_Blur")
        
        except:
            error_list.append(image_name)
            continue

[(0.53125, 0.4532407407407407, 0.36111111111111116, 0.5120370370370371, 0.0)]
[(0.46875, 0.4532407407407407, 0.36111111111111116, 0.5120370370370371, 0.0)]
[(0.53125, 0.4532407407407407, 0.36111111111111116, 0.5120370370370371, 0.0)]
[(0.078125, 0.7583333333333333, 0.0798611111111111, 0.17962962962962958, 0.0), (0.4076388888888889, 0.7305555555555555, 0.036111111111111094, 0.1685185185185185, 0.0)]
[(0.921875, 0.7583333333333333, 0.07986111111111116, 0.17962962962962958, 0.0), (0.5923611111111111, 0.7305555555555555, 0.03611111111111098, 0.1685185185185185, 0.0)]
[(0.078125, 0.7583333333333333, 0.0798611111111111, 0.17962962962962958, 0.0), (0.4076388888888889, 0.7305555555555555, 0.036111111111111094, 0.1685185185185185, 0.0)]
[(0.09131944444444444, 0.3666666666666666, 0.17847222222222223, 0.7240740740740741, 0.0), (0.5131944444444444, 0.46435185185185185, 0.6125, 0.9231481481481482, 0.0), (0.8618055555555556, 0.5, 0.2763888888888888, 0.9277777777777778, 0.0)]
[(0.9086805555555555, 0.

[(0.5107638888888889, 0.3819444444444444, 0.04513888888888884, 0.1972222222222222, 0.0), (0.5996527777777778, 0.375, 0.060416666666666674, 0.21666666666666667, 0.0), (0.2857638888888889, 0.48194444444444445, 0.1826388888888889, 0.5453703703703704, 0.0), (0.8666666666666667, 0.4375, 0.13194444444444442, 0.5138888888888888, 0.0), (0.4496527777777778, 0.3819444444444444, 0.03819444444444442, 0.1824074074074074, 0.0), (0.5722222222222222, 0.38101851851851853, 0.03888888888888886, 0.1842592592592593, 0.0), (0.14652777777777778, 0.3638888888888888, 0.04305555555555557, 0.137037037037037, 0.0), (0.20555555555555555, 0.35555555555555557, 0.036111111111111094, 0.12962962962962965, 0.0)]
[(0.4892361111111111, 0.3819444444444444, 0.04513888888888884, 0.1972222222222222, 0.0), (0.40034722222222213, 0.375, 0.060416666666666674, 0.21666666666666667, 0.0), (0.7142361111111111, 0.48194444444444445, 0.1826388888888889, 0.5453703703703704, 0.0), (0.1333333333333333, 0.4375, 0.13194444444444442, 0.513888

[(0.29930555555555555, 0.5194444444444445, 0.33472222222222225, 0.951851851851852, 0.0), (0.7236111111111112, 0.4324074074074074, 0.20138888888888895, 0.6296296296296297, 0.0)]
[(0.5263888888888889, 0.5421296296296296, 0.2597222222222222, 0.575, 0.0)]
[(0.4736111111111111, 0.5421296296296296, 0.2597222222222222, 0.575, 0.0)]
[(0.5263888888888889, 0.5421296296296296, 0.2597222222222222, 0.575, 0.0)]
[(0.5625, 0.3439814814814815, 0.25, 0.49537037037037035, 0.0)]
[(0.4375, 0.3439814814814815, 0.25, 0.49537037037037035, 0.0)]
[(0.5625, 0.3439814814814815, 0.25, 0.49537037037037035, 0.0)]
[(0.10208333333333333, 0.12638888888888888, 0.07916666666666666, 0.2046296296296296, 0.0), (0.6006944444444444, 0.22453703703703703, 0.1777777777777778, 0.3583333333333333, 0.0), (0.359375, 0.4425925925925926, 0.3201388888888888, 0.5333333333333333, 0.0), (0.9482638888888889, 0.2875, 0.10347222222222219, 0.4490740740740741, 0.0), (0.6631944444444444, 0.6935185185185185, 0.5, 0.6129629629629629, 0.0)]
[(0.8

[(0.3975694444444444, 0.5444444444444444, 0.7909722222222222, 0.9111111111111112, 0.0), (0.5635416666666667, 0.6287037037037037, 0.6034722222222222, 0.7425925925925927, 0.0)]
[(0.6024305555555556, 0.5444444444444444, 0.7909722222222222, 0.9111111111111112, 0.0), (0.4364583333333333, 0.6287037037037037, 0.6034722222222222, 0.7425925925925927, 0.0)]
[(0.3975694444444444, 0.5444444444444444, 0.7909722222222222, 0.9111111111111112, 0.0), (0.5635416666666667, 0.6287037037037037, 0.6034722222222222, 0.7425925925925927, 0.0)]
[(0.5010416666666667, 0.5277777777777778, 0.9979166666666666, 0.9444444444444444, 0.0)]
[(0.4989583333333333, 0.5277777777777778, 0.9979166666666666, 0.9444444444444444, 0.0)]
[(0.5010416666666667, 0.5277777777777778, 0.9979166666666666, 0.9444444444444444, 0.0)]
[(0.8895833333333334, 0.7847222222222222, 0.2069444444444445, 0.4305555555555556, 0.0)]
[(0.11041666666666666, 0.7847222222222222, 0.2069444444444445, 0.4305555555555556, 0.0)]
[(0.8895833333333334, 0.7847222222

[(0.7947916666666667, 0.5180555555555556, 0.40625, 0.9583333333333333, 0.0)]
[(0.3173611111111111, 0.41620370370370374, 0.16388888888888892, 0.7212962962962963, 0.0), (0.8100694444444444, 0.7185185185185186, 0.22152777777777777, 0.5574074074074074, 0.0)]
[(0.6826388888888889, 0.41620370370370374, 0.16388888888888897, 0.7212962962962963, 0.0), (0.1899305555555556, 0.7185185185185186, 0.22152777777777777, 0.5574074074074074, 0.0)]
[(0.3173611111111111, 0.41620370370370374, 0.16388888888888892, 0.7212962962962963, 0.0), (0.8100694444444444, 0.7185185185185186, 0.22152777777777777, 0.5574074074074074, 0.0)]
[(0.4482638888888889, 0.6513888888888889, 0.21875, 0.6879629629629631, 0.0), (0.5986111111111112, 0.6439814814814815, 0.1444444444444445, 0.5527777777777778, 0.0), (0.24166666666666667, 0.6587962962962963, 0.19444444444444448, 0.6824074074074074, 0.0), (0.7225694444444445, 0.625, 0.1659722222222222, 0.5666666666666667, 0.0)]
[(0.5517361111111111, 0.6513888888888889, 0.21875, 0.687962962

...

[(0.39444444444444443, 0.325, 0.18888888888888888, 0.42222222222222217, 0.0), (0.18506944444444445, 0.32731481481481484, 0.3104166666666667, 0.6453703703703704, 0.0), (0.5104166666666666, 0.36342592592592593, 0.10138888888888892, 0.43240740740740735, 0.0), (0.7135416666666666, 0.2375, 0.2826388888888889, 0.4212962962962963, 0.0)]
[(0.6055555555555555, 0.325, 0.18888888888888888, 0.42222222222222217, 0.0), (0.8149305555555555, 0.32731481481481484, 0.3104166666666667, 0.6453703703703704, 0.0), (0.48958333333333337, 0.36342592592592593, 0.10138888888888897, 0.43240740740740735, 0.0), (0.28645833333333337, 0.2375, 0.2826388888888889, 0.4212962962962963, 0.0)]
[(0.39444444444444443, 0.325, 0.18888888888888888, 0.42222222222222217, 0.0), (0.18506944444444445, 0.32731481481481484, 0.3104166666666667, 0.6453703703703704, 0.0), (0.5104166666666666, 0.36342592592592593, 0.10138888888888892, 0.43240740740740735, 0.0), (0.7135416666666666, 0.2375, 0.2826388888888889, 0.4212962962962963, 0.0)]
[(0.

[(0.7822916666666667, 0.5541666666666667, 0.4048611111111111, 0.623148148148148, 0.0)]
[(0.9298611111111111, 0.5083333333333333, 0.06527777777777777, 0.16111111111111115, 0.0), (0.6635416666666667, 0.7319444444444444, 0.11736111111111114, 0.46388888888888885, 0.0), (0.5618055555555556, 0.7615740740740742, 0.1791666666666667, 0.4638888888888889, 0.0), (0.2295138888888889, 0.5296296296296297, 0.0965277777777778, 0.20370370370370372, 0.0), (0.07013888888888889, 0.5444444444444443, 0.13472222222222224, 0.23333333333333328, 0.0), (0.28402777777777777, 0.7615740740740741, 0.2875, 0.47685185185185186, 0.0), (0.5673611111111111, 0.5509259259259259, 0.050000000000000044, 0.14814814814814814, 0.0)]
[(0.07013888888888886, 0.5083333333333333, 0.06527777777777777, 0.16111111111111115, 0.0), (0.33645833333333325, 0.7319444444444444, 0.11736111111111114, 0.46388888888888885, 0.0), (0.43819444444444444, 0.7615740740740742, 0.1791666666666667, 0.4638888888888889, 0.0), (0.7704861111111111, 0.5296296296

[(0.6774305555555555, 0.6087962962962963, 0.22013888888888888, 0.46388888888888896, 0.0), (0.09826388888888887, 0.6212962962962963, 0.1909722222222222, 0.5111111111111111, 0.0), (0.3975694444444444, 0.6106481481481482, 0.31736111111111104, 0.375, 0.0), (0.2725694444444444, 0.5851851851851851, 0.15763888888888888, 0.4574074074074075, 0.0), (0.2142361111111111, 0.30185185185185187, 0.14375, 0.1685185185185185, 0.0)]
[(0.32256944444444446, 0.6087962962962963, 0.22013888888888888, 0.46388888888888896, 0.0), (0.9017361111111111, 0.6212962962962963, 0.1909722222222222, 0.5111111111111111, 0.0), (0.6024305555555556, 0.6106481481481482, 0.317361111111111, 0.375, 0.0), (0.7274305555555556, 0.5851851851851851, 0.15763888888888888, 0.4574074074074075, 0.0), (0.7857638888888889, 0.30185185185185187, 0.14374999999999993, 0.1685185185185185, 0.0)]
[(0.6774305555555555, 0.6087962962962963, 0.22013888888888888, 0.46388888888888896, 0.0), (0.09826388888888887, 0.6212962962962963, 0.1909722222222222, 0.

[(0.6548611111111111, 0.5023148148148148, 0.6902777777777778, 0.9953703703703705, 0.0)]
[(0.3451388888888889, 0.5023148148148148, 0.6902777777777778, 0.9953703703703705, 0.0)]
[(0.6548611111111111, 0.5023148148148148, 0.6902777777777778, 0.9953703703703705, 0.0)]
[(0.7600694444444445, 0.5856481481481481, 0.28541666666666665, 0.8157407407407407, 0.0), (0.18124999999999997, 0.6037037037037037, 0.35, 0.7796296296296297, 0.0), (0.31319444444444444, 0.4111111111111111, 0.12638888888888888, 0.4277777777777778, 0.0), (0.41180555555555554, 0.38101851851851853, 0.1111111111111111, 0.3935185185185186, 0.0), (0.5, 0.36018518518518516, 0.11250000000000004, 0.2925925925925925, 0.0), (0.6451388888888889, 0.337037037037037, 0.09027777777777779, 0.2796296296296297, 0.0), (0.7013888888888888, 0.3958333333333333, 0.10277777777777775, 0.32685185185185184, 0.0)]
[(0.23993055555555554, 0.5856481481481481, 0.28541666666666665, 0.8157407407407407, 0.0), (0.8187500000000001, 0.6037037037037037, 0.35, 0.779629

[(0.30625, 0.6694444444444444, 0.19305555555555554, 0.6611111111111112, 0.0), (0.1451388888888889, 0.6398148148148148, 0.1375, 0.6851851851851851, 0.0), (0.5583333333333333, 0.5115740740740742, 0.18888888888888888, 0.34537037037037044, 0.0), (0.4322916666666667, 0.44074074074074077, 0.07013888888888886, 0.3222222222222222, 0.0), (0.49826388888888895, 0.30324074074074076, 0.04375000000000001, 0.11944444444444446, 0.0), (0.6368055555555555, 0.3421296296296296, 0.054166666666666696, 0.18425925925925923, 0.0), (0.7163194444444443, 0.5083333333333333, 0.12708333333333333, 0.4574074074074074, 0.0), (0.773263888888889, 0.3175925925925926, 0.05902777777777779, 0.1611111111111111, 0.0), (0.8663194444444444, 0.39861111111111114, 0.07986111111111116, 0.23796296296296293, 0.0), (0.703125, 0.32407407407407407, 0.053472222222222254, 0.16111111111111115, 0.0)]
[(0.69375, 0.6694444444444444, 0.19305555555555554, 0.6611111111111112, 0.0), (0.8548611111111111, 0.6398148148148148, 0.13750000000000007, 0.